In [332]:
import numpy as np
import pandas as pd
train = pd.read_csv('train.csv', sep='\t')
test = pd.read_csv('test.csv', sep='\t')
sample_submission = pd.read_csv('sample_submission.csv')

x_train = train["text"]
y_train = train["label"]
y_train = y_train.replace('label', 0 , regex=True)  #把label轉為0  df.replace or series.replace

x_test = test["text"]
y_test = sample_submission["label"]

In [333]:
from sklearn.feature_extraction.text import CountVectorizer
#reference: https://www.tutorialspoint.com/python_text_processing/python_remove_stopwords.htm
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stopwords_list = stopwords.words() #dataset包含了各國語言，所以不能只單獨使用english

cv = CountVectorizer(stop_words = stopwords_list)   #利用stop_words參數去把stop words去除掉
x_train = cv.fit_transform(train["text"])   #vector型態
x_test =  cv.transform(test["text"])   #vector型態

# fit_transform v.s. transform : https://medium.com/@maggieliao.cm04g/scikit-learn-%E6%95%B8%E6%93%9A%E9%A0%90%E8%99%95%E7%90%86-fit-transform-%E8%88%87transform-%E4%B9%8B%E5%B7%AE%E7%95%B0-3c7cc07c124f

print("x_train : ", x_train.shape)
print("x_test : ", x_test.shape)
print(x_train)
# print(x_test)

[nltk_data] Downloading package stopwords to /Users/danny/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
x_train :  (4987, 63250)
x_test :  (1247, 63250)
  (0, 23724)	1
  (0, 32553)	1
  (0, 56606)	2
  (0, 51038)	1
  (0, 38882)	1
  (0, 20124)	1
  (0, 57514)	1
  (0, 23732)	1
  (0, 33973)	1
  (0, 34039)	2
  (0, 7941)	6
  (0, 50550)	4
  (0, 20242)	1
  (0, 282)	1
  (0, 40053)	1
  (0, 9323)	3
  (0, 47443)	6
  (0, 16697)	3
  (0, 39450)	1
  (0, 237)	1
  (0, 628)	1
  (0, 39401)	1
  (0, 57801)	1
  (0, 37305)	1
  (0, 11263)	1
  :	:
  (4986, 3195)	1
  (4986, 3975)	2
  (4986, 44884)	1
  (4986, 41932)	1
  (4986, 44470)	1
  (4986, 309)	1
  (4986, 23373)	1
  (4986, 28347)	1
  (4986, 30641)	1
  (4986, 15952)	1
  (4986, 15511)	1
  (4986, 13528)	13
  (4986, 38691)	1
  (4986, 53261)	4
  (4986, 18087)	2
  (4986, 27625)	2
  (4986, 15950)	1
  (4986, 3585)	1
  (4986, 26424)	1
  (4986, 47155)	2
  (4986, 36324)	1
  (4986, 41181)	1
  (4986, 22336)	1
  (4986, 46457)	1
  (4986, 57425)	3


In [334]:
from sklearn.feature_extraction.text import TfidfTransformer    #TF-IDF計算個別term權重
tfidf = TfidfTransformer()
x_train_tfidf = tfidf.fit_transform(x_train)
x_test_tfidf = tfidf.transform(x_test)
print("x_train_tfidf 資料型態：", type(x_train_tfidf))
# print(x_test_tfidf)
print(x_train_tfidf)


x_train_tfidf 資料型態： <class 'scipy.sparse.csr.csr_matrix'>
  (0, 62123)	0.03621289735920745
  (0, 62003)	0.026467462361343258
  (0, 61994)	0.021996089826837876
  (0, 61708)	0.03503795667476213
  (0, 61597)	0.02578439087892961
  (0, 61109)	0.05229744895417865
  (0, 61016)	0.03618097484983312
  (0, 60796)	0.049898230593484846
  (0, 60591)	0.034029047189863604
  (0, 60475)	0.10405222327417042
  (0, 58499)	0.09886189701446695
  (0, 57875)	0.04911451728421352
  (0, 57865)	0.03511106321055882
  (0, 57801)	0.05304896023374879
  (0, 57768)	0.037037984413813424
  (0, 57514)	0.05185599418611214
  (0, 56641)	0.025601718234430252
  (0, 56606)	0.07551327353665142
  (0, 55584)	0.07961050259117623
  (0, 53445)	0.030043516710702357
  (0, 53166)	0.044974436477612394
  (0, 52357)	0.053596351358158406
  (0, 52173)	0.06890745747175894
  (0, 51452)	0.10405222327417042
  (0, 51277)	0.05298955910801573
  :	:
  (4986, 3912)	0.01616212067265448
  (4986, 3910)	0.016691048618657046
  (4986, 3585)	0.03097872511011

In [335]:

    def xgaboost():

        import xgboost as xgb

        d_train = xgb.DMatrix(x_train_tfidf, y_train)  # load a scipy.sparse array into DMatrix
        d_test = xgb.DMatrix(x_test_tfidf, y_test)
        print("d_train 資料型態：", type(d_train))

        # 建模 referece:https://blog.csdn.net/asialee_bird/article/details/94836962
        #
        param = {'silent': 0, 'eta': 0.3, 'max_depth': 6, 'objective': 'multi:softmax', 'num_class': 2, 'eval_metric': 'merror'}  #參數表
        evallist = [(d_train, 'train'), (d_test, 'test')]
        num_round = 100
        xgb_model = xgb.train(param, d_train, num_round,evallist)

        # 保存訓練模型
        # xgb_model.save_model('data/xgb_model')
        # xgb_model=xgb.Booster
        # (model_file='data/xgb_model') #讀取訓練好的xgboost模型


        from sklearn import metrics

        y_predict = xgb_model.predict(d_test)  # 模型預測
        label_all = ['真新聞','假新聞']
        confusion_matrix = metrics.confusion_matrix(y_test, y_predict)
        df = pd.DataFrame(confusion_matrix, columns=label_all)
        df.index = label_all
        print('準確率：', metrics.accuracy_score(y_test, y_predict))
        print('confusion_matrix:')
        print(df)
        print('分類報告:')
        print(metrics.classification_report(y_test, y_predict))

    def lightgbm(x_test, x_train_tfidf, y_train):
        from lightgbm import LGBMClassifier
        model = LGBMClassifier()
        y_train = y_train.astype('int')
        model = model.fit(x_train_tfidf, y_train)

        x_test = x_test.astype('float')
        y_predicted = model.predict(x_test)


        label_all = ['真新聞','假新聞']
        confusion_matrix = metrics.confusion_matrix(y_test, y_predicted)
        df = pd.DataFrame(confusion_matrix, columns=label_all)
        df.index = label_all
        print('準確率：', metrics.accuracy_score(y_test, y_predicted))
        print('confusion_matrix:')
        print(df)
        print('分類報告:')
        print(metrics.classification_report(y_test, y_predicted))
        # for kaggle output:
        # submission = pd.DataFrame({
        #                     "label": y_predicted
        #                  })
        # submission.to_csv('submission.csv', index=False)
    
    def GBDT(x_test, x_train_tfidf, y_train):
        from sklearn.ensemble import GradientBoostingClassifier
        clf = GradientBoostingClassifier(random_state=0)
        y_train = y_train.astype('int')
        clf = clf.fit(x_train_tfidf, y_train) 
        x_test = x_test.astype('float')
        y_predicted = clf.predict(x_test)

        label_all = ['真新聞','假新聞']
        confusion_matrix = metrics.confusion_matrix(y_test, y_predicted)
        df = pd.DataFrame(confusion_matrix, columns=label_all)
        df.index = label_all
        print('準確率：', metrics.accuracy_score(y_test, y_predicted))
        print('confusion_matrix:')
        print(df)
        print('分類報告:')
        print(metrics.classification_report(y_test, y_predicted))

In [336]:
xgaboost()

d_train 資料型態： <class 'xgboost.core.DMatrix'>
[21:57:03] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:516: 
Parameters: { silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[0]	train-merror:0.25887	test-merror:0.50441
[1]	train-merror:0.24063	test-merror:0.49559
[2]	train-merror:0.23120	test-merror:0.49800
[3]	train-merror:0.22920	test-merror:0.49559
[4]	train-merror:0.21777	test-merror:0.50120
[5]	train-merror:0.21155	test-merror:0.50201
[6]	train-merror:0.20373	test-merror:0.49960
[7]	train-merror:0.19290	test-merror:0.49880
[8]	train-merror:0.18689	test-merror:0.49880
[9]	train-merror:0.18107	test-merror:0.49719
[10]	train-merror:0.17686	test-merror:0.50281
[11]	train-merror:0.16744	test-merror:0.50281
[12]	train-merror:0.16322	test-merror:0.50762
[13]	train-

In [337]:
lightgbm(x_test, x_train_tfidf, y_train)

準確率： 0.49318364073777066
confusion_matrix:
     真新聞  假新聞
真新聞  381  249
假新聞  383  234
分類報告:
              precision    recall  f1-score   support

           0       0.50      0.60      0.55       630
           1       0.48      0.38      0.43       617

    accuracy                           0.49      1247
   macro avg       0.49      0.49      0.49      1247
weighted avg       0.49      0.49      0.49      1247



In [338]:
GBDT(x_test, x_train_tfidf, y_train)

準確率： 0.5140336808340016
confusion_matrix:
     真新聞  假新聞
真新聞  443  187
假新聞  419  198
分類報告:
              precision    recall  f1-score   support

           0       0.51      0.70      0.59       630
           1       0.51      0.32      0.40       617

    accuracy                           0.51      1247
   macro avg       0.51      0.51      0.49      1247
weighted avg       0.51      0.51      0.50      1247

